# Master Data Management

In [17]:
import pandas as pd
import requests
from sqlalchemy import create_engine

## Exportera

### Hämta från API

In [9]:
def get_df_from_api(endpoint):
    BASE_URL = "https://jsonplaceholder.typicode.com"
    
    response = requests.get(f"{BASE_URL}{endpoint}")
    data = response.json()
    return pd.DataFrame(data)

def format_address(row):
    address = row["address"]
    street = address["street"]
    suite = address["suite"]
    zipcode = address["zipcode"]
    city = address["city"]
    
    return f"{street} {suite}\n{zipcode} {city}"

df_users = get_df_from_api("/users")
df_posts = get_df_from_api("/posts")
df_comments = get_df_from_api("/comments")

df_users["formatted_address"] = df_users.apply(format_address, axis=1)
df_users.drop("company", axis=1, inplace=True)
df_users.drop("address", axis=1, inplace=True)

### Hämta från csv

In [10]:
df_products = pd.read_csv("olist_products_dataset.csv")
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


### Hämta från MySQL

In [23]:
connection_string = "mysql+mysqlconnector://root@localhost/northwind"

engine = create_engine(connection_string)

connection = engine.connect()

query = "SELECT * FROM customers"

df_customers = pd.read_sql(query, connection)

## Importera

### Till csv-filer

In [24]:
df_users.to_csv("df_users.csv")
df_posts.to_csv("df_posts.csv")
df_comments.to_csv("df_comments.csv")
df_products.to_csv("df_products.csv")
df_customers.to_csv("df_customers.csv")

### Till MySQL-databas

In [29]:
connection_string = "mysql+mysqlconnector://root@localhost/dmdd_mdm_db"

engine = create_engine(connection_string)

connection = engine.connect()

df_users.to_sql("df_users", connection, if_exists="replace", index=False)
df_posts.to_sql("df_posts", connection, if_exists="replace", index=False)
df_comments.to_sql("df_comments", connection, if_exists="replace", index=False)
df_customers.to_sql("df_customers", connection, if_exists="replace", index=False)

29

In [31]:
df_products.to_sql("df_products", connection, if_exists="replace", index=False, chunksize=50, method='multi')

32951